In [1]:
# Sample program for one scrip
from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi tws live
ib = IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [ ]:
%%time
import numpy as np
import pandas as pd
from itertools import product

# sd multiple for band
sigma = 2   # 2 sigma is about 95% probability

# market
exchange = 'NSE'

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

#... Get the scrip
symbol = 'ACC'
contract = Stock('ACC', 'NSE')
ib.qualifyContracts(contract)

#... Get the price from bar data 
# Last second's bar of historical with realtime updates
undPrice = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr='60 S', barSizeSetting='1 secs', 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)[-1].high

#... Get the volatility
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='MIDPOINT', useRTH=True, 
                     formatDate=1, keepUpToDate=True)
stDev = np.std(a=[b.high for b in bars], ddof=0)

#... Get the lot and margin
lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

#... Get the option chain
xk = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

#... Weed out unwanted SDs
sigma = 2   # 2 sigma is about 95% probability

safek = sorted([s for k in xk 
    for s in k.strikes 
    if (s < (undPrice - stDev*sigma)) | (s > (undPrice + stDev*sigma))])

tups = [(sorted(i.expirations), safek) for i in xk]

tups_xk = [p for tup in tups for p in product(*tup)]

#... Get the margins and lots

# make dataframe for the option contracts
df = pd.DataFrame(tups_xk, columns = ['expiry', 'strike'])

df['ibSymbol'] = symbol
df['exchange'] = exchange
df['lot'] = lot # miniumum margin
df['margin'] = margin # margin per lot

# puts for strike below undPrice and calls for strike above undPrice
df['right'] = np.where(df.strike < undPrice, 'P', 'C')

# list of options
opt_list = [Contract(symbol=symbol, secType='OPT', exchange=exchange, strike=strike,
       lastTradeDateOrContractMonth=expiry, right=right, )
for symbol, strike, expiry, right in zip(df.ibSymbol, df.strike, df.expiry, df.right)]

In [ ]:
# qualified contracts
contracts = [c for i in range(0, len(opt_list), 50)
             for c in ib.qualifyContracts(*opt_list[i: i+50])]

In [ ]:
tickers = ib.reqTickers(*contracts)
# ib.sleep(2)

In [65]:
dict_t = {ticker.contract.conId: ticker for ticker in tickers}

In [78]:
util.df([c.contract for c in tickers])

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,OPT,339611574,ACC,20181227,940.0,P,1,NSE,,INR,ACC18DEC940PE,ACC,False,,,,None,None
1,OPT,339361672,ACC,20181227,960.0,P,1,NSE,,INR,ACC18DEC960PE,ACC,False,,,,None,None
2,OPT,338818691,ACC,20181227,980.0,P,1,NSE,,INR,ACC18DEC980PE,ACC,False,,,,None,None
3,OPT,338818697,ACC,20181227,1000.0,P,1,NSE,,INR,ACC18DEC1000PE,ACC,False,,,,None,None
4,OPT,338002539,ACC,20181227,1020.0,P,1,NSE,,INR,ACC18DEC1020PE,ACC,False,,,,None,None
5,OPT,337010028,ACC,20181227,1040.0,P,1,NSE,,INR,ACC18DEC1040PE,ACC,False,,,,None,None
6,OPT,336637752,ACC,20181227,1060.0,P,1,NSE,,INR,ACC18DEC1060PE,ACC,False,,,,None,None
7,OPT,336324188,ACC,20181227,1080.0,P,1,NSE,,INR,ACC18DEC1080PE,ACC,False,,,,None,None
8,OPT,335897192,ACC,20181227,1100.0,P,1,NSE,,INR,ACC18DEC1100PE,ACC,False,,,,None,None
9,OPT,335897203,ACC,20181227,1120.0,P,1,NSE,,INR,ACC18DEC1120PE,ACC,False,,,,None,None


In [74]:
tickers[0]

Ticker(contract=Contract(secType='OPT', conId=339611574, symbol='ACC', lastTradeDateOrContractMonth='20181227', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18DEC940PE', tradingClass='ACC'), time=datetime.datetime(2018, 12, 17, 23, 29, 18, 363311, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.0, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], modelGreeks=OptionComputation(impliedVol=0.3762650841234649, delta=-2.9976021664879227e-15, optPrice=8.150926339662057e-14, pvDividend=0.0, gamma=2.61425080546738e-16, vega=5.739571509178659e-14, theta=-8.150926339662057e-14, undPrice=1502.0))

In [ ]:
asks = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='60 S', 
                      barSizeSetting='1 min', whatToShow='ASK', useRTH=True) 
          for c in contracts]

In [32]:
c = Contract(secType='OPT', conId=339611574, symbol='ACC', 
             lastTradeDateOrContractMonth='20181227', strike=163.0, right='P', 
             multiplier='1', exchange='NSE', currency='INR', 
             localSymbol='ACC18DEC940PE', tradingClass='ACC')

c = Contract(secType='OPT', conId=335422111, symbol='VEDL', 
             lastTradeDateOrContractMonth='20181227', strike=163.0, right='P', 
             multiplier='1', exchange='NSE', currency='INR', 
             localSymbol='VEDL18DEC163PE', tradingClass='VEDL')


In [33]:
ib.qualifyContracts(c)

[Contract(secType='OPT', conId=335422111, symbol='VEDL', lastTradeDateOrContractMonth='20181227', strike=163.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='VEDL18DEC163PE', tradingClass='VEDL')]

In [46]:
asks

NameError: name 'asks' is not defined

In [25]:
contracts[0]

Contract(secType='OPT', conId=339611574, symbol='ACC', lastTradeDateOrContractMonth='20181227', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18DEC940PE', tradingClass='ACC')

In [26]:
#... Get ask and bid prices
ib.reqHistoricalData(contract=contracts[0], endDateTime='', durationStr='1 D', 
                      barSizeSetting='1 day', whatToShow='BID', useRTH=True)

Error 162, reqId 361: Historical Market Data Service error message:HMDS query returned no data: ACC18DEC940PE@NSE Bid, contract: Contract(secType='OPT', conId=339611574, symbol='ACC', lastTradeDateOrContractMonth='20181227', strike=940.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC18DEC940PE', tradingClass='ACC')


[]

In [21]:
ib.ticker(contract=contracts[25])

In [ ]:
# order for getting margins
order = MarketOrder(action='SELL', totalQuantity=lot)

In [7]:
ib.whatIfOrder(contract=contracts[50], order=order)

OrderState(status='PreSubmitted', initMarginBefore='1.847183314E7', maintMarginBefore='1.477746652E7', equityWithLoanBefore='2.269576556E7', initMarginChange='94123.75', maintMarginChange='75298.99000000022', equityWithLoanChange='-23.929999999701977', initMarginAfter='1.856595689E7', maintMarginAfter='1.485276551E7', equityWithLoanAfter='2.269574163E7', commission=20.0, commissionCurrency='INR')

In [8]:
contracts[50]

Option(conId=343974440, symbol='ACC', lastTradeDateOrContractMonth='20190228', strike=1080.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC19FEB1080PE', tradingClass='ACC')

In [6]:
df_paisa[df_paisa.nseSymbol == 'ACC']

,nseSymbol,Mlot,SpMgn%,ExpMgn%,marginpct,SpMgnPerShare,ExpMgnPerShr,TotMgnPerShr,SpMgnPerLt,ExpMgnPerLt,TotMgnPerLt,ibSymbol
154,ACC,400,12.5,5.0,0.175,187.9,75.12,263.02,75160,30048.0,105208.0,ACC


In [ ]:
dict_ms = {contract.localSymbol: ib.whatIfOrder(contract=contract, order=order).maintMarginChange 
 for contract in contracts}
dict_ms

In [ ]:
dict_ms